In [1]:
# !rm -rf utils.py
#wget https://raw.githubusercontent.com/sevenfx/fastai_audio/master/notebooks/utils.py

In [8]:
%matplotlib inline
import os
import glob
import shutil
from pathlib import Path
from IPython.display import Audio
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
from utils import read_file, transform_path

ImportError: cannot import name 'read_file' from 'utils' (e:\python file\python37\lib\site-packages\utils-1.0.1-py3.7.egg\utils\__init__.py)

In [3]:
emotions = {
    '01': 'neutral',
    '02': 'calm',
    '03': 'happy',
    '04': 'sad',
    '05': 'angry',
    '06': 'fearful',
    '07': 'disgust',
    '08': 'surprised'
}
observed_emotions = ['neutral', 'happy', 'sad', 'angry']

In [4]:
if not os.path.exists('audios'):
    os.makedirs('audios')
if not os.path.exists('imgs'):
    os.makedirs('imgs')
for file in glob.glob("ravdess_data\Actor_*\*.wav"):
    audio_name = os.path.basename(file)
    path = os.path.join(os.path.dirname(os.path.abspath(file))[27:], audio_name)
    emotion = emotions[audio_name.split("-")[2]]
    if emotion in observed_emotions:
        shutil.copyfile(path, os.path.join('audios', audio_name))
AUDIO_DIR = Path('audios')
IMG_DIR = Path('imgs')

In [5]:
fnames = os.listdir(str(AUDIO_DIR))
len(fnames), fnames[:5]

(0, [])

In [ ]:
fn = fnames[94]
print(fn)
Audio(str(AUDIO_DIR/fn))

In [7]:
x, sr = read_file(fn, AUDIO_DIR)
x.shape, sr, x.dtype

NameError: name 'read_file' is not defined

In [ ]:
def log_mel_spec_tfm(fname, src_path, dst_path):
    x, sample_rate = read_file(fname, src_path)
    
    n_fft = 1024
    hop_length = 256
    n_mels = 40
    fmin = 20
    fmax = sample_rate / 2 
    
    mel_spec_power = librosa.feature.melspectrogram(x, sr=sample_rate, n_fft=n_fft, 
                                                    hop_length=hop_length, 
                                                    n_mels=n_mels, power=2.0, 
                                                    fmin=fmin, fmax=fmax)
    mel_spec_db = librosa.power_to_db(mel_spec_power, ref=np.max)
    dst_fname = dst_path / (fname[:-4] + '.png')
    plt.imsave(dst_fname, mel_spec_db)

In [ ]:
log_mel_spec_tfm(fn, AUDIO_DIR, IMG_DIR)
img = plt.imread(str(IMG_DIR/(fn[:-4] + '.png')))
plt.imshow(img, origin='lower');

In [ ]:
# transform_path(AUDIO_DIR, IMG_DIR, log_mel_spec_tfm, fnames=fnames, delete=True)
for fn in fnames:
    log_mel_spec_tfm(fn, AUDIO_DIR, IMG_DIR)
    img = plt.imread(str(IMG_DIR/(fn[:-4] + '.png')))

In [ ]:
os.listdir(str(IMG_DIR))[:10]

In [6]:
import fastai
from fastai.vision import *
fastai.__version__

ModuleNotFoundError: No module named 'fastai'

In [ ]:
digit_pattern = r'(\d+)-\d+-\d+-\d+-\d+\.png$'

In [ ]:
data = (ImageList.from_folder(IMG_DIR)
        .split_by_rand_pct(.2)
        .label_from_re(digit_pattern)
        .transform(size=(128,64))
        .databunch())
data.c, data.classes

In [ ]:
xs, ys = data.one_batch()
xs.shape, ys.shape

In [ ]:
xs.min(), xs.max(), xs.mean(), xs.std()

In [ ]:
data.show_batch(4, figsize=(5,9), hide_axis=False)

In [ ]:
learn = cnn_learner(data, models.resnet18, metrics=accuracy)
learn.fit_one_cycle(4)

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(4)

In [ ]:
learn.save('resnet-18-ser')

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_confusion_matrix(figsize=(10, 10), dpi=60)

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(2,max_lr=slice(1e-4 / 2 , 1e-2 / 2))

In [ ]:
learn.save('resnet-18-ser-finetune')

In [ ]:
interp_ft = ClassificationInterpretation.from_learner(learn)
interp_ft.plot_confusion_matrix(dpi=60, figsize=(10,10))

In [ ]:
interp_ft.plot_top_losses(k=12, heatmap=False)